# Database Overview

### Naming
The naming of the databases confines to the `iab_taxonomy-v2` found in `document_classification` repository. For example if you're look for the `Woodworking` table, then navigate to woodworking on the lookup JSON. You'll find that it's under 11.2.9, you simply have to access the table with table name `table_11_2_9` to get the all the documents.

### Columns
The columns in every database follows the same schema of `id, content, title, keywords, description`. Here, id refers to the url of the article or in some cases just a unique id to identify the document by. If any of the values weren't available then you'd notice a `N/A` value in the respective row.

# Import Packages

All the packages required to execute database queries are imported below

In [1]:
import psycopg2
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import json
from pandas.io.json import json_normalize

# Connect to database

In [2]:
def connect_to_db(database, hostname, port, userid, passwrd):
    # create string
    conn_string = "host={} port={} dbname={} user={} password={}".format(hostname,
                                                                         port,
                                                                         database,
                                                                         userid,
                                                                         passwrd)
    # connect to the database with the connection string
    conn = psycopg2.connect(conn_string)
    # commits all queries you execute
    conn.autocommit = True
    cursor = conn.cursor()
    return conn, cursor

# Get Table Values

In [3]:
def get_rows(cursor, table_name):
    # get all the rows from the table
    sqlSelect = 'SELECT * FROM {}'.format(table_name)
    cursor.execute(sqlSelect);
    rows = cursor.fetchall();
    return rows

In [4]:
conn, cursor = connect_to_db("postgres", 
                             "classification-articles.cjgj2uy1bapa.us-east-1.rds.amazonaws.com", 
                             "5432", 
                             "classifyarticles", 
                             "Hindsight2020")

In [5]:
table_name = 'table_13_1_14'
table_rows = get_rows(cursor, table_name)

In [6]:
table_rows[1]

(1,
 'https://www.medicalnewstoday.com/articles/ejection-fraction',
 'Ejection fraction refers to how much blood the heart pumps out during a contraction. An abnormal ejection fraction can be a sign of heart failure. Some people might need treatment to improve their ejection fraction.In this article, we discuss the ejection fraction in more detail, including what the results mean and how to improve this measurement. Ejection fraction is a measure of how well the heart is pumping blood around the body. The heart contracts and relaxes when it beats. During the contraction, it pushes blood out of large chambers called ventricles. These ventricles fill up with blood again as the heart relaxes.Ejection fraction is the percentage of blood that the heart pushes out during a contraction. The percentage typically refers to the ejection fraction of the left ventricle, the chamber of the heart that pumps blood to the head, arms, torso, abdomen, and legs. The ejection fraction is an important meas

**As it turns out**, there is a nice way to, in the framework that we are using, [to  print out all of the column names of your last query](https://stackoverflow.com/questions/17143132/python-psycopg2-postgres-select-columns-including-field-names/45050139).

In [65]:
len([ row for row in table_rows if row[-1] == "Heart\\ and\\ Cardiovascular\\ Diseases" ]) == len(table_rows)

True

**Thus**, each row in the table corresponds to the same **class node**!

In [58]:
[ row for (index, row) in enumerate(table_rows) if int(row[0]) == index ][15]

(26,
 'https://www.forbes.com/sites/forbestechcouncil/2018/08/02/the-meeting-animal/',
 "Introduction We hold meetings for every imaginable reason. The meeting is so fundamental to how we do work that we rarely think about why we meet. Meeting with other people is as natural as walking. Football players hold a meeting before every single play. As social creatures, we possess a profound ability to coordinate our actions in order to do work. There are some people who say we need to reinvent the meeting, and there are others who say we should get rid of meetings altogether. There are even some people that believe that information bots can eliminate the need for the meeting. While we love to meet, we also loathe it. As technology continues to disturb how we meet, we will use this paper to revisit the meeting and ask ourselves why we gather in the first place. Meetings are not going away any time soon. In a world that values speed and action, I'll take a minute here to revisit the essential

In [13]:
col_query = "SELECT * FROM table_13_1_14"
cursor.execute(col_query)

In [15]:
cursor.description

(Column(name='index', type_code=20),
 Column(name='id', type_code=25),
 Column(name='content', type_code=25),
 Column(name='title', type_code=25),
 Column(name='keywords', type_code=25),
 Column(name='description', type_code=25),
 Column(name='preprocessed_content', type_code=25),
 Column(name='label', type_code=25))

So it appears that each table in the database corresponds to a different category.

# Adding new computed values to Table

After computing new values if you want to append the those values which are currently in the form of a list/array, execute the below function. Note: keep in mind the indexs of the values must match correctly in order for the table to retain correct information

In [6]:
def add_column(conn, cursor, table_name, new_column_name, table_values, new_values):
    sqlStatement = 'ALTER TABLE '+table_name+' ADD COLUMN '+new_column_name+' TEXT;'
    cursor.execute(sqlStatement)  
    conn.commit() 
    for i in range(len(table_values)):
        cursor.execute('UPDATE test_table SET trials = %s WHERE id = %s', (new_values[i],table_values[i][0]));
        conn.commit()

In [7]:
p = ['newvfdv', 'colvfvdumn', 'insedfvfvdrt', 'hevdfvdfre', 'avfdvdlso']
add_column(conn, cursor, 'test_table', 'testing_algo', table_rows, p)

In [8]:
table_rows = get_rows(cursor, table_name)

In [9]:
table_rows

[('https://www.economist.com/news/united-states/159346-enterprise-immortalised?zid=319&amp;ah=17af09b0281b01505c226b1e574f5cc1',
  "DARLA MOORE left rural South Carolina and made a fortune on Wall Street as a specialist in bankruptcy financing. Last week the 43-year-old deal-maker went home to her alma mater, the University of South Carolina, when it named its business school after her in exchange for a donation of $25m. It was the latest example of a “naming opportunity” (or, as Slate on-line magazine brutally calls it, “monogrammed giving”).As America's thundering economy creates thousands of nouveaux riches, universities, colleges, museums, libraries and other non-profit groups are urging the newly rich to immortalise their names. Business schools do particularly well in the naming game, because their products tend to make more money than, say, philosophy graduates. Michael Price, a mutual-fund guru, gave $18m last year to the University of Oklahoma, and can now visit the Michael F.

# Finding table names

In [5]:
cursor.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
table_names = [t_name[0] for t_name in cursor.fetchall()]
table_names[:50:]

['table_5_8',
 'table_5_11',
 'table_27_2',
 'table_6',
 'table_8',
 'table_10',
 'table_11',
 'table_13',
 'table_16',
 'table_19',
 'table_17_7_2',
 'name_table',
 'test_table',
 'table_17_10_1',
 'table_17_10_2',
 'table_17_10_3',
 'table_17_10_4',
 'table_17_10',
 'table_17_9',
 'table_14_13',
 'table_14_3',
 'table_2_3',
 'table_2_4',
 'table_2_5',
 'table_2_6',
 'table_2_7',
 'table_2_8',
 'table_2_9',
 'table_2',
 'table_25_1_1',
 'table_25_8_3',
 'table_25_10',
 'table_3_3_13',
 'table_3_3_22',
 'table_3_3_29',
 'table_3',
 'table_24_33',
 'table_24_41',
 'table_24_43',
 'table_24_9',
 'table_15_32_5',
 'table_5',
 'table_27_1',
 'table_27_4',
 'table_27_7',
 'table_13_2',
 'table_13_3',
 'table_12_1',
 'table_12_10',
 'table_12_11']

In [6]:
table_names.remove("name_table")
table_names.remove("test_table")

In [9]:
import pickle
import os

In [10]:
current_path = os.getcwd()
rel_path_to_file = "../data/table_names.pkl"
full_path_to_file = os.path.join(current_path, rel_path_to_file)

In [12]:
pickle.dump( table_names, open( full_path_to_file, "wb" ) )

**NOTE** one important thing about the data that we have obtained. The text that can be found in the `preprocessed_content` column was obtained from that in the `content` column through the following means:
1. *All* of the text has been lower-cased.
2. *All* punctuation has been removed.
3. *All* stopwords have been removed.
4. *All* possesive versions of a word have converted to their non-possesive versions.
5. *All* word have been converted to their stems and then lemmas.
**For this reason**, the data that we currently have does *NOT* need to undergo any pre-processing!